In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install -q kagglehub torch torchvision scikit-learn opencv-python


In [4]:
import os
import kagglehub
import numpy as np
import torch
import torch.nn as nn

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [5]:
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
dataset_path = os.path.join(path, "Training")

print("Dataset path:", dataset_path)
print("Classes:", os.listdir(dataset_path))


Dataset path: /kaggle/input/brain-tumor-mri-dataset/Training
Classes: ['pituitary', 'notumor', 'meningioma', 'glioma']


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [7]:
dataset = ImageFolder(root=dataset_path, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

class_names = dataset.classes
print("Detected classes:", class_names)


Detected classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=None)     # ✅ NO DOWNLOAD
model.fc = nn.Identity()           # remove classifier
model = model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
features = []
labels = []

with torch.no_grad():
    for images, targets in loader:
        images = images.to(device)
        output = model(images)
        features.append(output.cpu().numpy())
        labels.append(targets.numpy())

X = np.vstack(features)   # ✅ NOW X EXISTS
y = np.hstack(labels)     # ✅ NOW y EXISTS

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (5712, 512)
y shape: (5712,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (4569, 512)
Test size: (1143, 512)


In [16]:
from sklearn.feature_selection import SelectKBest, chi2


In [17]:
selector = SelectKBest(score_func=chi2, k=200)
X_train_fs = selector.fit_transform(X_train, y_train)
X_test_fs = selector.transform(X_test)

print("After feature selection:", X_train_fs.shape)


After feature selection: (4569, 200)


In [19]:
from sklearn.ensemble import RandomForestClassifier


In [20]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_fs, y_train)

importances = rf.feature_importances_
top_idx = np.argsort(importances)[-100:]

X_train_hybrid = X_train_fs[:, top_idx]
X_test_hybrid = X_test_fs[:, top_idx]

print("Hybrid feature shape:", X_train_hybrid.shape)


Hybrid feature shape: (4569, 100)


In [21]:
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train_hybrid, y_train)

y_pred = clf.predict(X_test_hybrid)
acc = accuracy_score(y_test, y_pred)

print("Hybrid Model Accuracy:", acc)


Hybrid Model Accuracy: 0.8442694663167104
